# Proyecto
______________

API: https://dev.adalab.es/cinema/
Endpoint : https://dev.adalab.es/api/cinema/movies?year=2010&genre=Drama

In [1]:
#Importar librerías para su posterior uso en el código
import tqdm
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
import datetime
import json
from time import sleep
pd.set_option('display.max_columns', None) # para poder visualizar todas las columnas de los DataFrames
# Importar librerías de Selenium
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select

In [ ]:

# Lista de años buscados: 15 años
anio_lista = list(range(2009, 2025))

# Lista de géneros
generos_lista = ["Drama", "Comedy", "Action", "Fantasy", "Horror", "Mystery", "Romance", "Thriller"]

# Lista vacía en la cual almacenaremos los datos extrañidos de la API
datos_peliculas = []

# Bucle para iterar por año y género
for anio in anio_lista:
    for genero in generos_lista:
        # Determinar URL
        url_movies = f"https://dev.adalab.es/api/cinema/movies?year={anio}&genre={genero}"
        
        # Llamada a API
        res_url_movies = requests.get(url_movies)
        
        # Pausa para la API
        sleep(1) 
        
        # Verificar la espuesta de la API
        if res_url_movies.status_code == 200:
            try:
                json_url_general = res_url_movies.json()
                
                # Imprimir jason para verlo
                print(f"Response for {anio} - {genero}: {json_url_general}")
                
                # Verificar si la llamada tiene la info correcta
                if isinstance(json_url_general, dict) and 'results' in json_url_general:
                    movies = json_url_general['results']
                    
                    # Verificar que es lista
                    if isinstance(movies, list):
                        # Bucle por película para extraer info
                        for pelicula in movies:
                            id = pelicula.get("idOwner")
                            titulo = pelicula.get("title")
                            tipo = pelicula.get("type")
                            anio = pelicula.get("year")
                            genero = pelicula.get("genre")
                            
                            # Añadir info a la lista anteriormente vacía
                            datos_peliculas.append((id, titulo, tipo, anio, genero))
                    else:
                        print("Expected 'results' to be a list.")
                else:
                    print("Expected JSON structure not found.")
            except json.JSONDecodeError:
                print("Error decoding JSON response.")
        else:
            print(f"Error: {res_url_movies.status_code} - {res_url_movies.reason}")

# Convertir a pandas frame
df_peliculas = pd.DataFrame(datos_peliculas)

# Salvar la busqueda en un documento con formato CSV
df_peliculas.to_csv("movies_last_15_years.csv", index=False)

print("Data extraction complete and saved to movies_last_15_years.csv.")


In [ ]:
print(datos_peliculas)
print(type(datos_peliculas))

In [ ]:
# Obtiene los nombres en la posición 2 (0,1,2) que es en donde está el type
nombres_tipos = set(pelicula [2] for pelicula in datos_peliculas)
print("Los tipos dentro de posición 2 son:", nombres_tipos)

# Crear listas vacías para cada tipo de película en minúsculas y añadir info con list comprehension
tv_mini_series_lista = [pelicula for pelicula in datos_peliculas if pelicula[2].replace('_', ' ') == "TV Mini Series"]
movie = [pelicula for pelicula in datos_peliculas if pelicula[2].replace('_', ' ') == "Movie"]
tv_series = [pelicula for pelicula in datos_peliculas if pelicula[2].replace('_', ' ') == "TV Series"]
short = [pelicula for pelicula in datos_peliculas if pelicula[2].replace('_', ' ') == "Short"]
tv_episode = [pelicula for pelicula in datos_peliculas if pelicula[2].replace('_', ' ') == "TV Episode"]
video = [pelicula for pelicula in datos_peliculas if pelicula[2].replace('_', ' ') == "Video"]
tv_movie = [pelicula for pelicula in datos_peliculas if pelicula[2].replace('_', ' ') == "TV Movie"]
video_game = [pelicula for pelicula in datos_peliculas if pelicula[2].replace('_', ' ') == "Video Game"]

print(movie)


In [ ]:
# Iniciar el navegador
driver = webdriver.Chrome()
print("Abriendo Google")
# Maximizar la ventana del navegador
driver.maximize_window()
print("Pantalla maximizada")
sleep(3)

# Abrir página web
driver.get("https://www.imdb.com/")
print("Accediendo a la web")
sleep(3)

# Aceptar cookies
driver.find_element("css selector", "#__next > div > div > div.sc-jrcTuL.bPmWiM > div > button.icb-btn.sc-bcXHqe.sc-dkrFOg.sc-iBYQkv.dcvrLS.ddtuHe.dRCGjd").click()
print("Cookies aceptadas")
sleep(5)

# Diccionario vacío en dónde se almacenarán los datos
datos_imdb = {"puntuacion":[], "direccion":[], "guionista":[], "argumento":[], "nombre_pelicula":[], "id_pelicula":[]}

# Obtiene los nombres en la posición 2 (0,1,2) que es en donde está el type
nombres_tipos = set(pelicula [2] for pelicula in datos_peliculas)
print("Los tipos dentro de posición 2 son:", nombres_tipos)

# Crear listas vacías para cada tipo de película en minúsculas y añadir info con list comprehension
tv_mini_series_lista = [pelicula for pelicula in datos_peliculas if pelicula[2].replace('_', ' ') == "TV Mini Series"]
movie = [pelicula for pelicula in datos_peliculas if pelicula[2].replace('_', ' ') == "Movie"]
tv_series = [pelicula for pelicula in datos_peliculas if pelicula[2].replace('_', ' ') == "TV Series"]
short = [pelicula for pelicula in datos_peliculas if pelicula[2].replace('_', ' ') == "Short"]
tv_episode = [pelicula for pelicula in datos_peliculas if pelicula[2].replace('_', ' ') == "TV Episode"]
video = [pelicula for pelicula in datos_peliculas if pelicula[2].replace('_', ' ') == "Video"]
tv_movie = [pelicula for pelicula in datos_peliculas if pelicula[2].replace('_', ' ') == "TV Movie"]
video_game = [pelicula for pelicula in datos_peliculas if pelicula[2].replace('_', ' ') == "Video Game"]

# Iterar sobre los IDs de las películas
id_pelicula = [pelicula[0] for pelicula in movie]


for id_pelicula in id_pelicula:
    url_pelicula = f"https://www.imdb.com/title/{id_pelicula}/"

    # Navegar a la página de la película usando el ID
    driver.get(url_pelicula)
    sleep(5)

    try:
        # Extraer el nombre de la película
        titulo = driver.find_element("css selector", "h1 span").text
        datos_imdb["nombre_pelicula"].append(titulo)
        datos_imdb["id_pelicula"].append(id_pelicula)
        print(f"Titulo: {titulo}")
        
        # Extraer la puntuación
        try:
            puntuacion = driver.find_element("css selector", "span.sc-eb51e184-1.ljxVSS").text
            datos_imdb["puntuacion"].append(puntuacion)
            print(f"Puntuacion: {puntuacion}")
        except:
            datos_imdb["puntuacion"].append("N/A")
            print("Puntuación no encontrada")
        
        # Extraer la dirección
        try:
            directores = driver.find_element("CSS selector", '#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-afa4bed1-0.iMxoKo > section > div:nth-child(5) > section > section > div.sc-491663c0-4.gxWIhN > div.sc-491663c0-6.etrlfo > div.sc-491663c0-10.iaQXlA > section > div.sc-1f50b7c-3.gLpgJQ > div > ul > li:nth-child(1) > div > ul > li > a').text
            datos_imdb["direccion"].append(directores)
            print(f"Dirección: {directores}")
        except:
            datos_imdb["direccion"].append("N/A")
            print("Dirección no encontrada")

        #Extraer guionista
        try:
            guionista = driver.find_element("CSS selector", '#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-afa4bed1-0.iMxoKo > section > div:nth-child(5) > section > section > div.sc-491663c0-4.ILcwq > div.sc-491663c0-6.bvzCJs > div.sc-491663c0-11.hFWIYv > div.sc-1f50b7c-2.cpicUu > div > ul > li:nth-child(2) > div > ul').text
            datos_imdb["guionista"].append(guionista)
            print(f"Guionista: {guionista}")
        except:
            datos_imdb["guionista"].append("N/A")
            print("Guionista no encontrado")

        #Extraer argumento
        try:
            argumento = driver.find_element("CSS selector", 'p span').text
            datos_imdb["argumento"].append(argumento)
            print(f"Argumento: {argumento}")
        except:
            datos_imdb["argumento"].append("N/A")
            print("Argumento no encontrado")
        

    except Exception as e:
        print(f"Error al procesar la película con ID '{id_pelicula}': {e}")
        datos_imdb["nombre_pelicula"].append("N/A")
        datos_imdb["id_pelicula"].append(id_pelicula)
        datos_imdb["puntuacion"].append("N/A")
        datos_imdb["direccion"].append("N/A")
        # (Añadir "N/A" o valores por defecto para otros campos si es necesario)

print(datos_imdb)
# Cerrar el navegador
driver.quit()

- **EXTRACCION ACTORES** 

Instertar aquí

In [ ]:
# URL de la página de Wikipedia
url_oscar = "https://es.wikipedia.org/wiki/Premios_%C3%93scar"

# Realizar la solicitud HTTP a la página
res_url_oscar = requests.get(url_oscar)

# Verificar si la solicitud fue exitosa
if res_url_oscar.status_code == 200:
    # Parsear el contenido HTML con BeautifulSoup
    sopa_oscar = BeautifulSoup(res_url_oscar.content, "html.parser")

    # dict vacío en dónde se almacenarán los datos encontrados
    dict_datos_oscar = {"Ceremonia": [], "Mejor película": [], "Mejor director": [], "Mejor actor": [], "Mejor actriz": []}

    # Extraemos los datos de, únicamente la tabla dada.
    tabla_oscar = sopa_oscar.select_one("#mw-content-text .wikitable")
    # css selector COMPLETO: #mw-content-text > div.mw-content-ltr.mw-parser-output > table.wikitable
    # El selector se ha simplificado en caso de haber tablas anidadas. Y PORQUE CON EL COMPLETO NO NOS SALÍA ...... 
    
    # Crear una lista vacía para almacenar las tuplas
    lista_tuplas_oscar = []

    # Verificar que la tabla haya sido extraída correctamente
    if tabla_oscar:
        # Iterar sobre la tabla buscando las filas (Inspeccionando la web, vemos que las filas están en tr)
        filas = tabla_oscar.find_all('tr')
        
        # Iterando sobre las filas encontradas dentro de la tabla
        for fila in filas[1:]:  # La primera fila es el encabezdo que no necesitamos.
            
            celdas = fila.find_all('td') # Inspeccionando la web, vemos que en td, está el contenido de las celdas
            if len(celdas) >= 6: # Hay que revisar la cantidad de posiciones que tenemos
                
                fecha_divida = ceremonia.split() # dividir el texto

                fecha_anio = fecha_divida[-1]
                anio = int(fecha_anio)


                if anio >= 2000:
                    # Extraer contenido de celdas por posición/ index 
                    ceremonia = celdas[1].text.strip() #extraemos por index y sólo el texto y sin espacios extras
                    mejor_pelicula = celdas[2].text.strip()
                    mejor_director = celdas[3].text.strip()
                    mejor_actor = celdas[4].text.strip()
                    mejor_actriz = celdas[5].text.strip()
                    
                    # Añadir los datos al diccionario
                    dict_datos_oscar["Ceremonia"].append(fecha_divida[-1]) #que solo nos traiga la última posición ya que coresponde al año
                    dict_datos_oscar["Mejor película"].append(mejor_pelicula)
                    dict_datos_oscar["Mejor director"].append(mejor_director)
                    dict_datos_oscar["Mejor actor"].append(mejor_actor)
                    dict_datos_oscar["Mejor actriz"].append(mejor_actriz)

                    # Creamos una tupla con los datos extraídos de cada fila de la tabla y se añaden a la lista de tuplas ("lista_tuplas_oscar")
                    tupla = (fecha_divida, mejor_pelicula, mejor_director, mejor_actor, mejor_actriz)
                    lista_tuplas_oscar.append(tupla)

            # Comprobando que el display sea el máximo para mostrar mejor el data frame
            pd.set_option('display.max_rows', 100)  # Cambia el número máximo de filas mostradas
            pd.set_option('display.max_columns', 10)  # Cambia el número máximo de columnas mostradas
            pd.set_option('display.width', 1000)  # Cambia el ancho máximo de la pantalla

            # Convertir el diccionario en un DataFrame de pandas
            df_oscar = pd.DataFrame(dict_datos_oscar)

            # Mostrar el DataFrame
            print(df_oscar)

         # Iterar y printar cada tupla de la lista de tuplas ("lista_tuplas_oscar")
        print("Lista de tuplas:")
        for tupla in lista_tuplas_oscar:
            print(tupla)

    else:
        print("No se ha encontrado la tabla 'wikitable'")
else:
    print(f"Error al acceder a la página: {res_url_oscar.status_code}")
